# TRAINING MODELS

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../INTENT RECOGNITION')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../NAME ENTITY RECOGNITION')))


In [ ]:
from ner import NamedEntityRecognition
from intent_recognition import IntentRecognition

In [ ]:
train_config = { 'use_class_weights':True}
hyperparams = {'vocab_size': 500, 'embedding_dim': 768, 'epochs': 10, 'batch_size': 32}

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, LSTM, GRU, Bidirectional, Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D, Dense, Dropout


In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), merge_mode="ave"))
model.add(GlobalMaxPooling1D())

basic = IntentRecognition(
            model, 
            hyperparams, 
            training_times=1, 
            train_config=train_config, 
            verbosing=0, 
            name="Bidirectional",
            save_results=False
        )
basic.train_model()

In [ ]:
tokenized = np.array(basic.tokenizer.texts_to_sequences(["I want a to eat a steak to Boston please"]))

In [ ]:
probs = basic.model.predict(tokenized)
_predicted_labels = np.argmax(probs, axis=1)
predicted_labels = basic.label_encoder.inverse_transform(_predicted_labels)

In [ ]:
predicted_labels

In [ ]:
model = Sequential()
model.add(Bidirectional(GRU(128, return_sequences=True)))

ner_model = NamedEntityRecognition(
            model, 
            hyperparams, 
            training_times=1, 
            train_config=train_config, 
            verbosing=0, 
            name="Bidirectional",
            save_results=False
        )
ner_model.train_model()

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenized2 = np.array(ner_model.tokenizer.texts_to_sequences(["i need a ticket to go to san diego"]))

tokenized2

In [ ]:
max_length = ner_model.model.input_shape[1] 
tokenized2_padded = pad_sequences(tokenized2, maxlen=max_length, padding="post")

In [ ]:
probs = ner_model.model.predict(tokenized2_padded)

In [ ]:
probs = ner_model.model.predict(tokenized2_padded)
_predicted_labels = np.argmax(probs, axis=2)
_predicted_labels
predicted_labels = [ner_model.label_encoder.inverse_transform(a) for a in _predicted_labels]
print(predicted_labels)

# CLASS

In [ ]:
# Import necessary packages
import numpy as np

class DialogSystem:
    def __init__(self, intent_model, ner_model):
        self.intent_model = intent_model
        self.ner_model = ner_model
    
    def preprocess_input(self, user_input):
        return np.array(self.intent_model.tokenizer.texts_to_sequences([user_input]))

    def get_intent(self, user_input):
        tokenized_input = self.preprocess_input(user_input)
        tokenized2_padded = pad_sequences(tokenized_input, maxlen=max_length, padding="post")
        probs = self.intent_model.model.predict(tokenized2_padded)
        predicted_label = np.argmax(probs, axis=1)
        return self.intent_model.label_encoder.inverse_transform(predicted_label)[0]

    def get_entities(self, user_input):
        tokenized_input = np.array(self.ner_model.tokenizer.texts_to_sequences([user_input]))
        max_length = self.ner_model.model.input_shape[1] 
        tokenized2_padded = pad_sequences(tokenized_input, maxlen=max_length, padding="post")
        probs = self.ner_model.model.predict(tokenized2_padded)
        predicted_labels = np.argmax(probs, axis=2)[0]
        entities = []
        current_entity = None
        for i, label in enumerate(predicted_labels[:len(user_input.split())]):
            label_str = self.ner_model.label_encoder.inverse_transform([label])[0]
            if label_str != 'O': 
                if label_str.startswith("B-"):
                    if current_entity:
                        entities.append(current_entity)
                    current_entity = {"entity": label_str, "text": user_input.split()[i]}
                elif label_str.startswith("I-"): 
                    if current_entity:
                        current_entity["text"] += " " + user_input.split()[i]  
            else:
                if current_entity:
                    entities.append(current_entity) 
                    current_entity = None
        
        if current_entity: 
            entities.append(current_entity)

        return entities

    def generate_response(self, intent, entities):
        """
        Generate a response based on the identified intent and entities.
        """
        city_name = None
        state_name = None
        
        for entity in entities:
            if "city_name" in entity["entity"]:
                city_name = entity["text"]
            elif "state_name" in entity["entity"]:
                state_name = entity["text"]
        
        if intent == "flight":
            if city_name:
                return f"Looking up flight information to {city_name}."
            elif state_name:
                return f"Looking up flight information to {state_name}."
            else:
                return "Please specify a destination for the flight."

        elif intent == "airline":
            return "Retrieving airline information for you."

        elif intent == "airport":
            if city_name or state_name:
                location = city_name if city_name else state_name
                return f"Looking up airport information near {location}."
            else:
                return "Please specify a location for the airport."

        else:
            return "I'm sorry, I didn't understand your request."

    def handle_user_input(self, user_input):
        """
        Process user input, identify intent and entities, and generate a response.
        """
        intent = self.get_intent(user_input)
        entities = self.get_entities(user_input)
        response = self.generate_response(intent, entities)
        return response


# USE

In [ ]:
dialog_system = DialogSystem(basic, ner_model)

def chat_with_bot():
    print("Welcome to the ATIS-based dialog system! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        print("You: ", user_input)
        if user_input.lower() == 'exit':
            print("Bot: Goodbye!")
            break
        response = dialog_system.handle_user_input(user_input)
        print("Bot:", response)

chat_with_bot()


In [ ]:

def chat_with_bot():
    print("Welcome to the dialog system! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Bot: Goodbye!")
            break
        response = dialog_system.handle_user_input(user_input)
        print("Bot:", response)

chat_with_bot()